[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Fuenfgeld/2022TeamBDataEngineeringBC/blob/main/Workshop.ipynb)

# SQLite Workshop

### Bitte führen Sie die nächsten beiden Codeabschnitte aus

In [3]:
#Load Libraries
import sqlite3
import pandas as pd

# Load GitHub repo
%cd /content/
%rm -r ./2022TeamBDataEngineeringBC
!git clone https://github.com/Fuenfgeld/2022TeamBDataEngineeringBC.git
%cd ./2022TeamBDataEngineeringBC/

[WinError 2] Das System kann die angegebene Datei nicht finden: '/content/'
C:\Users\Sofia Henninger\Documents\Master\DAF\2022TeamBDataEngineeringBC


UsageError: Line magic function `%rm` not found.


In [5]:
# Connect to Database
conn = sqlite3.connect('sqlite_workshop.db')
cursor = conn.cursor()
print("Opened database successfully");

# Delete table if exists 
cursor.execute("DROP TABLE IF EXISTS food")
cursor.execute("DROP TABLE IF EXISTS orders")
cursor.execute("DROP TABLE IF EXISTS animals")
cursor.execute("DROP TABLE IF EXISTS customer")
cursor.execute("DROP TABLE IF EXISTS customer_buys_food")
cursor.execute("DROP TABLE IF EXISTS employee")
cursor.execute("DROP TABLE IF EXISTS sales")
cursor.execute("DROP TABLE IF EXISTS food_new")
cursor.execute("DROP TABLE IF EXISTS customer_new")
cursor.execute("DROP TABLE IF EXISTS employee_new")
cursor.execute("DROP TABLE IF EXISTS Liga")
cursor.execute("DROP TABLE IF EXISTS Verein")
cursor.execute("DROP TABLE IF EXISTS Spieler")

# Create tables
cursor.execute('''
		CREATE TABLE IF NOT EXISTS Liga (
            L_name nvarchar(80) primary key,
            Anzahl_Vereine int
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS Verein (
			V_name nvarchar(80) primary key,
            Liga nvarchar(80),
            Anzahl_Meisterschaften int,
            foreign key(Liga) references Liga(L_name)

			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS Spieler (
			S_name nvarchar(80) primary key,
            Verein nvarchar(80),
            foreign key(Verein) references Verein(V_name)

			)
               ''')

# fill tables for Part 1
cursor.execute('''
		INSERT INTO Liga (L_name,Anzahl_Vereine)
            VALUES
            ("1. Bundesliga", 18),
            ("2. Bundesliga", 18), 
            ("3. Bundesliga",20)

			
               ''')

cursor.execute('''
		INSERT INTO Verein (V_name, Liga, Anzahl_Meisterschaften)
            VALUES
            ("1. FC Bayern Muenchen", "1. Bundesliga", 32),
            ("2. FC Kaiserslautern", "3. Bundesliga", 4), 
            ( "SV Waldhof Mannheim", "3. Bundesliga", 0)

			
               ''')
cursor.execute('''
		INSERT INTO Spieler (S_name, Verein)
            VALUES
            ("Manuel Neuer","1. FC Bayern Muenchen"),
            ("Matheo Raab", "1. FC Kaiserslautern"), 
            ("Timo Koenigsmann", "SV Waldhof Mannheim")

			
               ''')


Opened database successfully


# Aufgabe zu Teil 1


## 1.1 Ersetze die Fragezeichen, um eine gültige PRAGMA Funktion zu nutzen
## 1.2 Zeichne anschließend ein ER-Diagramm

In [6]:
# Tabellennamen herausfinden
print(pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn))


      name
0     Liga
1   Verein
2  Spieler


In [7]:
# Private Keys herausfinden
print(pd.read_sql_query("???('???')", conn))
print("----------------------------------------")
print(pd.read_sql_query("???('???')", conn))
print("----------------------------------------")
print(pd.read_sql_query("???('???')", conn))

DatabaseError: Execution failed on sql '???('???')': near "?": syntax error

In [8]:
# Foreign keys herausfinden
print(pd.read_sql_query("PRAGMA ???('???')", conn))
print("----------------------------------------")
print(pd.read_sql_query("PRAGMA ???('???')", conn))
print("----------------------------------------")
print(pd.read_sql_query("PRAGMA ???('???')", conn))

DatabaseError: Execution failed on sql 'PRAGMA ???('???')': near "?": syntax error

In [9]:
print(pd.read_sql_query("SELECT * FROM Liga", conn))
print("----------------------------------------")
print(pd.read_sql_query("SELECT * FROM Verein", conn))
print("----------------------------------------")
print(pd.read_sql_query("SELECT * FROM Spieler", conn))

Empty DataFrame
Columns: [L_name, Anzahl_Vereine]
Index: []
----------------------------------------
Empty DataFrame
Columns: [V_name, Liga, Anzahl_Meisterschaften]
Index: []
----------------------------------------
Empty DataFrame
Columns: [S_name, Verein]
Index: []


## Bitte führen Sie diesen Teil aus 

In [10]:
# Daten einlesen
food = pd.read_csv('tables/initial_tables/Futter.csv', sep=";")
customer = pd.read_csv('tables/initial_tables/Kunden.csv', sep=";")
animals = pd.read_csv('tables/initial_tables/Tiere.csv', sep=";")
orders = pd.read_csv('tables/initial_tables/Order.csv', sep=";")
employee = pd.read_csv('tables/initial_tables/Verkaeufer.csv', sep=";")
customer_buys_food = pd.read_csv('tables/initial_tables/Customer_buys_Food.csv', sep=";")


# Tabellen erstellen
cursor.execute('''
		CREATE TABLE IF NOT EXISTS customer (
			customer_ID int primary key,
            name nvarchar(50),
			postal_code nvarchar(5),
			city nvarchar(50)
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS employee (
			employee_ID int primary key,
            name nvarchar(50),
            gender nvarchar(1),
			wage int
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS food (
			food_ID int primary key,
            name nvarchar(50),
			price_per_kg float,
            producer nvarchar(50),
            origin nvarchar(50)
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS animals (
            name nvarchar(50) primary key,
			food int,
			foreign key(food) references food(food_ID)
			)
               ''') 
cursor.execute('''
		CREATE TABLE IF NOT EXISTS orders (
            order_Nr int primary key,
            date Date,
            transact int
			)
               ''')


cursor.execute('''
		CREATE TABLE IF NOT EXISTS customer_buys_food (
            C_ID int,
            F_ID int,
			amount int,
            O_ID int,
            E_ID int,
            foreign key(C_ID) references customer(customer_ID)
			foreign key(F_ID) references food(food_ID)
            foreign key(O_ID) references orders(order_Nr)
            foreign key(E_ID) references employee(employee_ID)
			)
               ''') 

# Insert Dataframe in Table
customer.to_sql('customer', conn, if_exists='append', index=False)
animals.to_sql('animals', conn, if_exists='append', index=False)
food.to_sql('food', conn, if_exists='append', index=False)
orders.to_sql('orders', conn, if_exists='append', index=False)
employee.to_sql('employee', conn, if_exists='append', index=False)
customer_buys_food.to_sql('customer_buys_food', conn, if_exists='append', index=False)

12

### Hiermit werden die Tabelleninhalte angezeigt

In [11]:
print("Tabelle customer")
print(pd.read_sql_query("SELECT * FROM customer", conn))
print("--------------------------------------------------------")
print("Tabelle food")
print(pd.read_sql_query("SELECT * FROM food", conn))
print("--------------------------------------------------------")
print("Tabelle animals")
print(pd.read_sql_query("SELECT * FROM animals", conn))
print("--------------------------------------------------------")
print("Tabelle orders")
print(pd.read_sql_query("SELECT * FROM orders", conn))
print("--------------------------------------------------------")
print("Tabelle employee")
print(pd.read_sql_query("SELECT * FROM employee", conn))
print("--------------------------------------------------------")
print("Tabellen customer_buys_food")
print(pd.read_sql_query("SELECT * FROM customer_buys_food", conn))
print("--------------------------------------------------------")

Tabelle customer
   customer_ID       name postal_code          city
0            1    Mueller       68723    Oftersheim
1            2    Schmidt       68766    Hockenheim
2            3  Schneider       68526     Ladenburg
3            4    Fischer       68753     Waghäusel
4            5      Weber       68799     Reilingen
5            6      Meyer       68549     Ilvesheim
6            7     Wagner       69221    Dossenheim
7            8     Becker       69251       Gaiberg
8            9     Schulz       69151  Neckargemünd
9           10   Hoffmann       68526     Ladenburg
--------------------------------------------------------
Tabelle food
   food_ID              name  price_per_kg          producer       origin
0        1       Hundefutter          4.00   Wau Dosenfutter  Deutschland
1        2      Katzenfutter          8.00  Miau Dosenfutter        China
2        3  Getreidemischung         10.00  Jerry's Getreide  Deutschland
3        4      Pferdefutter          0.65   

# Aufgaben zu Teil 2

## ER-Diagramm

[<img src="https://raw.githubusercontent.com/Fuenfgeld/2022TeamBDataEngineeringBC/main/Presentation/Bilder/nur_tabellen.draw_io.drawio_3.png">](https://raw.githubusercontent.com/Fuenfgeld/2022TeamBDataEngineeringBC/main/Presentation/Bilder/nur_tabellen.draw_io.drawio_3.png)

## SUBQUERIES

#### Beispiel: Welche Kunden (C_ID) haben Futter gekauft, das in China hergestellt wurde?

In [12]:
print(pd.read_sql_query('''
    SELECT C_ID 
    FROM customer_buys_food
    WHERE F_ID =
       (SELECT food_ID FROM food WHERE origin == 'China')
    ''', conn))

   C_ID
0     3
1     7
2     3


### Aufgabe 2.1: Welche Bestellungen (O_ID) wurden von den Verkäufern abgewickelt, die mehr als 2000 verdienen?

In [13]:
# Ersetze die Fragezeichen 
print(pd.read_sql_query('''
     SELECT ???? 
     FROM ????
     WHERE ???? ??
       (SELECT ???? FROM ???? WHERE ???? > 2000)
     ''', conn))

DatabaseError: Execution failed on sql '
     SELECT ???? 
     FROM ????
     WHERE ???? ??
       (SELECT ???? FROM ???? WHERE ???? > 2000)
     ': near "?": syntax error

### Aufgabe 2.2: Welche Kunden (name) haben Futter von weiblichen Verkäufern verkauft bekommen?

In [14]:
# Schreibe die Abfrage an die Stelle der Fragezeichen
# Tipp: Es müssen zwei ineinander verschachtelte Subqueries verwendet werden.
print(pd.read_sql_query('''
    
    ????
        )
    ''', conn))

DatabaseError: Execution failed on sql '
    
    ????
        )
    ': near "?": syntax error

# Aufgaben zu Teil 3

## Bitte führen Sie diesen Teil aus 

In [16]:
# Sternschema erstellen
#cursor.execute('''PRAGMA foreign_keys = OFF;''')  # disable foreign keys
#conn.commit()
cursor.execute('''CREATE TABLE IF NOT EXISTS food_new(
                  food_id INT PRIMARY KEY,
                  name TEXT,
                  price_per_kg FLOAT,
                  producer TEXT,
                  origin TEXT,
                  animal TEXT
                  );''')

cursor.execute('''CREATE TABLE IF NOT EXISTS customer_new(
                  cust_id INT PRIMARY KEY NOT NULL UNIQUE,
                  name TEXT,
                  postal_code nvarchar(5),
                  city TEXT
                  );''')

cursor.execute('''CREATE TABLE IF NOT EXISTS employee_new(
                  empl_id INT PRIMARY KEY NOT NULL UNIQUE,
                  name TEXT,
                  gender nvarchar(1),
                  wage INT
                  );''')
conn.commit()

# Tabellen mit Daten füllen
cursor.execute('''INSERT INTO food_new(food_id, name, price_per_kg, producer, origin, animal)
                  SELECT food.food_ID, food.name, food.price_per_kg, food.producer, food.origin, animals.name
                  FROM food
                  INNER JOIN animals ON food.food_ID = animals.food;
                  ''')
conn.commit()
    
cursor.execute('''INSERT INTO employee_new(empl_id, name, gender, wage)
                  SELECT employee_ID, name, gender, wage
                  FROM employee;
                  ''')    
    
cursor.execute('''INSERT INTO customer_new(cust_id, name, postal_code, city)
                  SELECT customer_ID, name, postal_code, city
                  FROM customer;
                  ''')

conn.commit()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS sales(
        order_nr  INT PRIMARY KEY NOT NULL UNIQUE,
        transact INT,
        date DATE,
        custumer_id int, 
        food_id INT,
        amount INT,
        employee_id INT,
            foreign key(food_id) references food_new(food_id),
            foreign key(custumer_id) references customer_new(cust_id),
            foreign key(employee_id) references employee_new(empl_id)
        );''')

cursor.execute('''
    INSERT INTO sales( order_nr, transact, date, custumer_id, food_id, amount,employee_id )
    SELECT order_Nr,transact, date, C_ID, food_ID, amount, E_ID 
    FROM orders
    LEFT JOIN 
        customer_buys_food ON orders.order_Nr = customer_buys_food.O_ID
    LEFt JOIN 
        food ON customer_buys_food.F_ID = food.food_ID;
    ''')
cursor.execute(''' PRAGMA table_info(sales);''')
records = cursor.fetchall()

### Hiermit werden die Inhalte der Dimensionstabellen angezeigt

In [17]:
print("Tabelle customer_new")
print(pd.read_sql_query("SELECT * FROM customer_new", conn))
print("--------------------------------------------------------")
print("Tabelle food_new")
print(pd.read_sql_query("SELECT * FROM food_new", conn))
print("--------------------------------------------------------")
print("Tabelle employee_new")
print(pd.read_sql_query("SELECT * FROM employee", conn))
print("--------------------------------------------------------")

Tabelle customer_new
   cust_id       name postal_code          city
0        1    Mueller       68723    Oftersheim
1        2    Schmidt       68766    Hockenheim
2        3  Schneider       68526     Ladenburg
3        4    Fischer       68753     Waghäusel
4        5      Weber       68799     Reilingen
5        6      Meyer       68549     Ilvesheim
6        7     Wagner       69221    Dossenheim
7        8     Becker       69251       Gaiberg
8        9     Schulz       69151  Neckargemünd
9       10   Hoffmann       68526     Ladenburg
--------------------------------------------------------
Tabelle food_new
   food_id              name  price_per_kg          producer       origin  \
0        1       Hundefutter          4.00   Wau Dosenfutter  Deutschland   
1        2      Katzenfutter          8.00  Miau Dosenfutter        China   
2        3  Getreidemischung         10.00  Jerry's Getreide  Deutschland   
3        4      Pferdefutter          0.65          Hottehue       Ta

### Hiermit werden die Inhalte der Faktentabelle angezeigt

In [18]:
print(pd.read_sql_query("SELECT * FROM sales", conn))

    order_nr  transact        date  custumer_id  food_id  amount  employee_id
0          1         2  2021-04-22            2        4       2            2
1          2         1  2021-04-25            3        2       3            4
2          3         1  2021-05-03            4        1       5            5
3          4         3  2021-06-06            1        3       5           10
4          5         7  2021-07-22            5        5       2            7
5          6         4  2021-10-23            6        1       1            3
6          7         5  2022-01-20            9        4       2            6
7          8         5  2022-02-15            2        3       2            3
8          9         3  2022-03-21            8        5       1            1
9         10         6  2022-03-22            7        2       3            3
10        11         6  2022-03-23           10        3       2            3
11        12         3  2022-03-23            3        2       6

## JOINS

#### Beispiel: Welcher Verkäufer hat am meisten Umsatz erwirtschaftet?

In [20]:
print(pd.read_sql_query('''
    SELECT 
    employee.name as employee, employee.employee_ID, customer_buys_food.amount, food.price_per_kg,
    SUM(food.price_per_kg*customer_buys_food.amount) as sum_sales
    FROM employee
    LEFT JOIN 
        customer_buys_food  ON employee.employee_ID = customer_buys_food.E_ID
    INNER JOIN 
        food ON customer_buys_food.F_ID = food.food_ID 
    GROUP BY employee.name
    ORDER BY sum_sales DESC
    ''', conn))

    employee  employee_ID  amount  price_per_kg  sum_sales
0       Koch            3       1          4.00      116.0
1      Braun           10       5         10.00       50.0
2    Richter            4       3          8.00       24.0
3      Klein            5       5          4.00       20.0
4  Schroeder            7       2          2.30        4.6
5   Schaefer            1       1          2.30        2.3
6       Wolf            6       2          0.65        1.3
7      Bauer            2       2          0.65        1.3


### Aufgabe 3.1: Welcher Verkäufer hat am häufigsten Futter verkauft? Zeigen Sie seinen Namen, seine ID und sein Gehalt an.

In [15]:
print(pd.read_sql_query('''
    SELECT employee.employee_ID, employee.name, employee.wage, COUNT(customer_buys_food.O_ID) as sum_sales
    FROM customer_buys_food
    LEFT JOIN 
        employee ON employee.employee_ID = customer_buys_food.E_ID
    GROUP BY customer_buys_food.E_ID
    ORDER BY sum_sales DESC
    ''', conn))

   employee_ID       name  wage  sum_sales
0            3       Koch  2500          5
1           10      Braun  2000          1
2            7  Schroeder  1750          1
3            6       Wolf  2200          1
4            5      Klein  1750          1
5            4    Richter  3000          1
6            2      Bauer  1200          1
7            1   Schaefer  1000          1


### Aufgabe:

In [19]:
print(pd.read_sql_query('''
    SELECT * FROM sales
    WHERE employee_id IN
       (SELECT empl_id FROM employee_new WHERE wage > 2000)
    ''', conn))

   order_nr  transact        date  custumer_id  food_id  amount  employee_id
0         2         1  2021-04-25            3        2       3            4
1         6         4  2021-10-23            6        1       1            3
2         7         5  2022-01-20            9        4       2            6
3         8         5  2022-02-15            2        3       2            3
4        10         6  2022-03-22            7        2       3            3
5        11         6  2022-03-23           10        3       2            3
6        12         3  2022-03-23            3        2       6            3


### Close connection

In [ ]:
#conn.close()